In [44]:
import numpy as np
import pandas as pd
import nltk
import time
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JAYANTH.M\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JAYANTH.M\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
df = pd.read_csv(r'C:\Users\JAYANTH.M\Downloads\movies.csv',na_values="nan")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [46]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [47]:
df['genres'].fillna('mode',inplace=True)
df['keywords'].fillna('mode',inplace=True)
df['runtime'].fillna('mean',inplace=True)

In [48]:
temp_genres = []
temp_keywords = []
for index,row in df.iterrows():
    temp_genres.append(nltk.tokenize.word_tokenize(row['genres'].lower()))
    temp_keywords.append(nltk.tokenize.word_tokenize(row['keywords'].lower()))

df['genres'] = temp_genres
df['keywords'] = temp_keywords 

In [49]:
stop_words = set(stopwords.words('english'))
token_pop_genres = []
token_pop_keywords = []
for index,row in df.iterrows():
  for i in row['genres']:
    if i not in token_pop_genres:
      token_pop_genres.append(i)
  for i in stop_words:
    while i in row['keywords']:
      row['keywords'].remove(i)
    if i not in token_pop_keywords:
      token_pop_keywords.append(i)

v_genres = []
v_keywords = []
for index,row in df.iterrows():
  temp = []
  for i in token_pop_genres:
    count =0
    for j in row['genres']:
      if i == j:
        count+=1
    temp.append(count)
  v_genres.append(temp)

for index,row in df.iterrows():
  temp = []
  for i in token_pop_keywords:
    count =0
    for j in row['keywords']:
      if i == j:
        count+=1
    temp.append(count)
  v_keywords.append(temp)

df['genres'] = v_genres
df['keywords'] = v_keywords

In [50]:
temp = []
for index,row in df.iterrows():
  if row['vote_average'] < 6.2:
    temp.append(0)
  else:
    temp.append(1)

df['class'] = temp

In [51]:
temp = []
for index,rows in df.iterrows():
  temp.append((rows['popularity']-df['popularity'].min())/(df['popularity'].max()-df['popularity'].min()))

df['popularity'] = temp

In [52]:
feature_vector = []
for i,r in df.iterrows():
  temp=[r['popularity']]
  feature_vector.append(r['genres']+r['keywords']+temp)

df['vectors'] = feature_vector

In [53]:
df.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    0
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
class                      0
vectors                    0
dtype: int64

In [54]:
df.drop(['homepage'], axis=1,inplace=True)
df.drop(['overview'], axis=1,inplace=True)
df.drop(['director'], axis=1,inplace=True)
df.drop(['cast'], axis=1,inplace=True)
df.drop(['tagline'], axis=1,inplace=True)
df.drop(['release_date'], axis=1,inplace=True)
df.isnull().sum()

index                   0
budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
popularity              0
production_companies    0
production_countries    0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title                   0
vote_average            0
vote_count              0
crew                    0
class                   0
vectors                 0
dtype: int64

In [55]:
X = df['vectors']
Y = df['class']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.4,random_state=0)

### Classifications 

### KNN - k nearest neighbors

In [56]:
neigh = KNeighborsClassifier(n_neighbors=3)
start = time.time()
neigh.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is",neigh.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)
start = time.time()
neigh.predict(list(X_test))
end = time.time()

The excecution time of training is 0.052343130111694336
The accuracy is 0.6436004162330905
The excecution time of testing is 0.2510550022125244


### Perceptron

In [57]:
clf = Perceptron()
start = time.time()
clf = clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.06218099594116211
The accuracy is  0.6347554630593132
The excecution time of testing is 0.045932769775390625


### Multi Layer Perception

In [58]:
MLP = MLPClassifier()
start = time.time()
MLP.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",MLP.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 4.872934579849243
The accuracy is  0.6940686784599376
The excecution time of testing is 0.0877540111541748


C:\Users\JAYANTH.M\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Support Vector Machine

### Kernel Function = Linear

In [59]:
clf = svm.SVC(kernel='linear')
start = time.time()
clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.8214433193206787
The accuracy is  0.668054110301769
The excecution time of testing is 0.32748961448669434


### Kernel Function = Poly

In [60]:
clf = svm.SVC(kernel='poly')
start = time.time()
clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.7901227474212646
The accuracy is  0.658168574401665
The excecution time of testing is 0.3477010726928711


### Kernel Function = rbf

In [61]:
clf = svm.SVC(kernel='rbf')
start = time.time()
clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.828345775604248
The accuracy is  0.6644120707596254
The excecution time of testing is 0.990924596786499


### Kernel Function = Sigmoid

In [62]:
clf = svm.SVC(kernel='sigmoid')
start = time.time()
clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.6529595851898193
The accuracy is  0.5759625390218522
The excecution time of testing is 0.24764084815979004


### Decision Tree Classfier

### Criterion Measure = Gini

In [63]:
model = tree.DecisionTreeClassifier()
start = time.time()
model = model.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",model.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)
print(model.get_depth())

The excecution time of training is 0.04587888717651367
The accuracy is  0.6378772112382934
The excecution time of testing is 0.024216890335083008
30


### Criterion Measure = Entropy

In [64]:
model = tree.DecisionTreeClassifier(criterion="entropy")
start = time.time()
model = model.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",model.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)
print(model.get_depth())

The excecution time of training is 0.0578465461730957
The accuracy is  0.6404786680541104
The excecution time of testing is 0.024883747100830078
39


### Random Forest Classifier

In [65]:
clf = RandomForestClassifier()
start = time.time()
clf = clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.5713009834289551
The accuracy is  0.6519250780437045
The excecution time of testing is 0.059471845626831055


### Naive Bayes

In [66]:
model = GaussianNB()
start = time.time()
model = model.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",model.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.058397769927978516
The accuracy is  0.6690946930280958
The excecution time of testing is 0.04057765007019043


### Logistic Regression

In [67]:
clf = LogisticRegression()
start = time.time()
clf = clf.fit(list(X_train),Y_train)
end = time.time()
print("The excecution time of training is",end-start)
start = time.time()
print("The accuracy is ",clf.score(list(X_test),Y_test))
end = time.time()
print("The excecution time of testing is",end-start)

The excecution time of training is 0.06921911239624023
The accuracy is  0.68210197710718
The excecution time of testing is 0.05323386192321777


### Regression

In [68]:
X = df['budget']
Y = df['revenue']
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X, Y, test_size=0.05,random_state=0)

In [69]:
regr = linear_model.LinearRegression()
start = time.time()
regr.fit(X1_train.values.reshape(-1,1), Y1_train)
end = time.time()
print("The time taken to train the model is",end-start)
start = time.time()
Y1_pred = regr.predict(X1_test.values.reshape(-1,1))
end = time.time()
print("The time taken to predict the testing dataset",end-start)
print("Coefficient of determination: %.2f" % r2_score(Y1_test, Y1_pred))

The time taken to train the model is 0.00112152099609375
The time taken to predict the testing dataset 0.0
Coefficient of determination: 0.51


### Decision Tree Regressor

In [70]:
regtree = DecisionTreeRegressor() 
start = time.time()
regtree.fit(X1_train.values.reshape(-1,1), Y1_train)
end = time.time()
print("The time taken to train the Decision Tree Regressor model is",end-start)
start = time.time()
Y1_pred = regtree.predict(X1_test.values.reshape(-1,1))
end= time.time()
print("The time taken to predict the testing dataset",end-start)
print("Coefficient of determination: %.2f" % r2_score(Y1_test, Y1_pred))

The time taken to train the Decision Tree Regressor model is 0.005198001861572266
The time taken to predict the testing dataset 0.0010046958923339844
Coefficient of determination: 0.51


### KNN Regressor

In [71]:
KNNR = KNeighborsRegressor(n_neighbors=3)
start = time.time()
KNNR.fit(X1_train.values.reshape(-1,1), Y1_train)
end = time.time()
print("The time taken to train the KNN regressor model is",end-start)
start = time.time()
Y1_pred = KNNR.predict(X1_test.values.reshape(-1,1))
end = time.time()
print("The time taken to predict the testing dataset",end-start) 
print("Coefficient of determination: %.2f" % r2_score(Y1_test, Y1_pred))

The time taken to train the KNN regressor model is 0.0029916763305664062
The time taken to predict the testing dataset 0.002051115036010742
Coefficient of determination: 0.46
